# Calibration QAQC
- Visualize calibration routes and compare to the chosen and shortest routes
- Trip-specific impedance routing to see if chosen route can be found
- Test different objective functions
- Try using coordinates in case the map matched trace is incorrect

In [1]:
from pathlib import Path
import time
import pandas as pd
import geopandas as gpd
import numpy as np
import pickle
import networkx as nx
from stochopy.optimize import minimize
from tqdm import tqdm
import similaritymeasures
import random
import matplotlib.pyplot as plt
from shapely.ops import Point, MultiLineString, LineString
from importlib import reload
import datetime
from scipy.spatial.distance import directed_hausdorff
np.set_printoptions(suppress=True)

from bikewaysim.paths import config, stadia_toner, maptiler_streets
from bikewaysim.impedance_calibration import stochastic_optimization, speedfactor
from bikewaysim.network import modeling_turns
from bikewaysim.routing import rustworkx_routing_funcs
from bikewaysim.impedance_calibration import optimization_viz, utils


from step_1_calibration_experiments import all_calibrations, full_model

# Import relevant files

In [2]:
links, turns, length_dict, geo_dict, turn_G = rustworkx_routing_funcs.import_calibration_network(config)
with (config['calibration_fp']/'ready_for_calibration_stats.pkl').open('rb') as fh: # has loss values for shortest path
    match_results = pickle.load(fh)
full_ods = utils.match_results_to_ods_w_year(match_results)
# for adding coordinates to the viz part
with (config['cycleatl_fp']/"rdp.pkl").open('rb') as fh:
    coords = pickle.load(fh)

with (config['cycleatl_fp']/"trips_2.pkl").open('rb') as fh:
    trips = pickle.load(fh)
with (config['cycleatl_fp']/"users_2.pkl").open('rb') as fh:
    users = pickle.load(fh)
trips.set_index('tripid',inplace=True)
users.set_index('userid',inplace=True)
trips = trips.loc[list(match_results.keys())]
users = users.loc[users.index.isin(set(list(trips['userid'])))]
user_map = trips['userid'].to_dict()

# First an overview of how to use the QAQC functions

In [3]:
# retrieves the calibration result for each model calibrated
calibration_results_fps = (config['calibration_fp']/'results').glob('*.pkl')
calibration_results = {}
for calibration_results_fp in calibration_results_fps:
    with calibration_results_fp.open('rb') as fh:
        calibration_results[calibration_results_fp.stem] = pickle.load(fh)

# retrieves post routing and loss values results for each model calibrated
post_calibration_fps = (config['calibration_fp']/'loss').glob('*.pkl')
post_calibration = {}
for post_calibration_fp in post_calibration_fps:
    with post_calibration_fp.open('rb') as fh:
        post_calibration[post_calibration_fp.stem] = pickle.load(fh)

## Visualize a random trip

In [4]:
# get 
pool = list(match_results.keys())
not_in_pool = []
for key, post_calibration_dict in post_calibration.items():
    [x for x in pool if x not in post_calibration_dict.keys()]

In [5]:
post_calibration.keys()

dict_keys(['100,jaccard_buffer_mean,0', '100,jaccard_buffer_mean,1', '100,jaccard_buffer_mean,2', '101,jaccard_buffer_mean,0', '101,jaccard_buffer_mean,1', '101,jaccard_buffer_mean,2', '1013,jaccard_buffer_mean,0', '1013,jaccard_buffer_mean,1', '1016,jaccard_buffer_mean,0', '1016,jaccard_buffer_mean,1', '1017,jaccard_buffer_mean,0', '1017,jaccard_buffer_mean,1', '102,jaccard_buffer_mean,0', '102,jaccard_buffer_mean,1', '102,jaccard_buffer_mean,2', '1022,jaccard_buffer_mean,0', '1022,jaccard_buffer_mean,1', '1024,jaccard_buffer_mean,0', '1024,jaccard_buffer_mean,1', '1026,jaccard_buffer_mean,0', '1026,jaccard_buffer_mean,1', '1027,jaccard_buffer_mean,0', '1027,jaccard_buffer_mean,1', '1028,jaccard_buffer_mean,0', '1028,jaccard_buffer_mean,1', '1029,jaccard_buffer_mean,0', '1029,jaccard_buffer_mean,1', '103,jaccard_buffer_mean,0', '103,jaccard_buffer_mean,1', '103,jaccard_buffer_mean,2', '1030,jaccard_buffer_mean,0', '1030,jaccard_buffer_mean,1', '1035,jaccard_buffer_mean,0', '1035,jacca

In [6]:
#
calibration_results.keys()

dict_keys(['100,jaccard_buffer_mean,0', '100,jaccard_buffer_mean,1', '100,jaccard_buffer_mean,2', '101,jaccard_buffer_mean,0', '101,jaccard_buffer_mean,1', '101,jaccard_buffer_mean,2', '1013,jaccard_buffer_mean,0', '1013,jaccard_buffer_mean,1', '1016,jaccard_buffer_mean,0', '1016,jaccard_buffer_mean,1', '1017,jaccard_buffer_mean,0', '1017,jaccard_buffer_mean,1', '102,jaccard_buffer_mean,0', '102,jaccard_buffer_mean,1', '102,jaccard_buffer_mean,2', '1022,jaccard_buffer_mean,0', '1022,jaccard_buffer_mean,1', '1024,jaccard_buffer_mean,0', '1024,jaccard_buffer_mean,1', '1026,jaccard_buffer_mean,0', '1026,jaccard_buffer_mean,1', '1027,jaccard_buffer_mean,0', '1027,jaccard_buffer_mean,1', '1028,jaccard_buffer_mean,0', '1028,jaccard_buffer_mean,1', '1029,jaccard_buffer_mean,0', '1029,jaccard_buffer_mean,1', '103,jaccard_buffer_mean,0', '103,jaccard_buffer_mean,1', '103,jaccard_buffer_mean,2', '1030,jaccard_buffer_mean,0', '1030,jaccard_buffer_mean,1', '1035,jaccard_buffer_mean,0', '1035,jacca

In [7]:
# just bootstrap
bootstrap_results = {x:item for x, item in calibration_results.items() if 'bootsample' in x}
bootstrap_routes = {x:item for x, item in post_calibration.items() if 'bootsample' in x}

In [8]:
bootstrap_routes.keys()

dict_keys(['bootsample_0,validation,0', 'bootsample_1,validation,0', 'bootsample_10,validation,0', 'bootsample_100,validation,0', 'bootsample_101,validation,0', 'bootsample_102,validation,0', 'bootsample_103,validation,0', 'bootsample_104,validation,0', 'bootsample_105,validation,0', 'bootsample_106,validation,0', 'bootsample_107,validation,0', 'bootsample_108,validation,0', 'bootsample_109,validation,0', 'bootsample_11,validation,0', 'bootsample_110,validation,0', 'bootsample_111,validation,0', 'bootsample_112,validation,0', 'bootsample_113,validation,0', 'bootsample_114,validation,0', 'bootsample_115,validation,0', 'bootsample_116,validation,0', 'bootsample_117,validation,0', 'bootsample_118,validation,0', 'bootsample_119,validation,0', 'bootsample_12,validation,0', 'bootsample_120,validation,0', 'bootsample_121,validation,0', 'bootsample_122,validation,0', 'bootsample_123,validation,0', 'bootsample_124,validation,0', 'bootsample_125,validation,0', 'bootsample_126,validation,0', 'boo

In [ ]:
# #TODO add route attributes used for each line
# #TODO add 
# #BUG legend doesn't seem to be updating correctly
# reload(stochastic_optimization)

# tripid = random.choice(pool)
# tripid = 34346
# userid = user_map[tripid]
# print('tripid:',tripid,'| userid:',userid)
# # tripid = 32136 not enough difference between streets to matter
# reload(stochastic_optimization)
# reload(optimization_viz)
# # subset_post_calibration = {key:item for key,item in post_calibration.items() if user_map[tripid] == int(key.split('_')[0])}
# print(trips.loc[tripid,['avg_speed_mph','trip_type','description']])
# print(users.loc[trips.at[tripid,'userid']])
# m = optimization_viz.visualize_three(tripid,full_set,post_calibration,calibration_results,geo_dict,coords,config['projected_crs_epsg'],stadia_toner)
# m.save(Path.home()/'Downloads/test_viz.html')
# m

In [12]:
#TODO add route attributes used for each line
#TODO add 
#BUG legend doesn't seem to be updating correctly
reload(stochastic_optimization)

tripid = random.choice(pool)
tripid = 34346
userid = user_map[tripid]
print('tripid:',tripid,'| userid:',userid)
# tripid = 32136 not enough difference between streets to matter
reload(stochastic_optimization)
reload(optimization_viz)
# subset_post_calibration = {key:item for key,item in post_calibration.items() if user_map[tripid] == int(key.split('_')[0])}
print(trips.loc[tripid,['avg_speed_mph','trip_type','description']])
print(users.loc[trips.at[tripid,'userid']])
m = optimization_viz.visualize_three(tripid,match_results,bootstrap_routes,bootstrap_results,geo_dict,coords,config['projected_crs_epsg'],stadia_toner)
m.save(Path.home()/'Downloads/test_viz.html')
m

tripid: 34346 | userid: 1654
avg_speed_mph              10.104984
trip_type                    Commute
description      after breaking down
Name: 34346, dtype: object
age                               25-34
gender                             Male
income              $100,000 or greater
ethnicity                         White
cycling_freq     Several times per week
rider_history             Several years
rider_type         Enthused & confident
count                               291
Name: 1654, dtype: object


ValueError: max() arg is an empty sequence

## Try out custom impedance

In [ ]:
betas_tup = (
    {'col': '2lpd', 'type': 'link', 'range': [0, 3], 'beta': 2},
    {'col': '3+lpd', 'type': 'link', 'range': [0, 3], 'beta': 5},
    {'col': '(30,40] mph', 'type': 'link', 'range': [0, 3], 'beta': 1},
    {'col': '(40,inf) mph', 'type': 'link', 'range': [0, 3], 'beta': 3},
    {'col': '[4k,10k) aadt', 'type': 'link', 'range': [0, 3], 'beta': 2},
    {'col': '[10k,inf) aadt', 'type': 'link', 'range': [0, 3], 'beta': 9},
    {'col': '[4,6) grade', 'type': 'link', 'range': [0, 3], 'beta': 2},
    {'col': '[6,inf) grade', 'type': 'link', 'range': [0, 3], 'beta': 9},
    {'col': 'bike lane', 'type': 'link', 'range': [-1, 0], 'beta': -0.5},
    {'col': 'cycletrack', 'type': 'link', 'range': [-1, 0], 'beta': -0.5},
    {'col': 'multi use path', 'type': 'link', 'range': [-1, 0], 'beta': -0.3},
    {'col': 'unsig_crossing', 'type': 'turn', 'range': [0, 2], 'beta': 0.1}
)
custom_route = optimization_viz.create_custom_route(betas_tup,tripid,match_results,links,turns,turn_G,length_dict,geo_dict)
m = optimization_viz.visualize_three(tripid,match_results,post_calibration,calibration_results,geo_dict,coords,config['projected_crs_epsg'],stadia_toner,custom_route)
m

In [ ]:
custom_route

## Or see what the route output would be for one or every model between two ODs

## Idenfity trips that pass through a circle
One area with trouble is the Stone Mountain Trail because the map matched route often uses Dekalb instead. This leads to some trips having a lower than expected overlap.

In [ ]:
reload(stochastic_optimization)

In [ ]:
freedom_pkwy = (2237899.09,1372338.05)
freedom_pkwy = stochastic_optimization.retrieve_geos(*freedom_pkwy,modeled_results,links)
print(len(freedom_pkwy))

In [ ]:
smt = (2250499.40,1369121.80)
smt = stochastic_optimization.retrieve_geos(*smt,modeled_results,links)
print(len(smt))
tenth_st = (2233722.10,1375729.08)
tenth_st = stochastic_optimization.retrieve_geos(*tenth_st,modeled_results,links)
print(len(tenth_st))
wylie_st = (2237751.33,1365098.89)
wylie_st = stochastic_optimization.retrieve_geos(*wylie_st,modeled_results,links)
print(len(wylie_st))

In [ ]:
tripid = random.choice(freedom_pkwy)
reload(stochastic_optimization)
stochastic_optimization.basic_three_viz(tripid,modeled_results,links.crs,length_dict,geo_dict,tile_info_dict)